In [1]:
# adapted based on https://www.tensorflow.org/tutorials/keras/regression
import matplotlib.pyplot as plt
import numpy as np
np.set_printoptions(precision=3, suppress=True)
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

2024-02-27 23:01:06.182159: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-02-27 23:01:06.226378: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-27 23:01:06.226410: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-27 23:01:06.226443: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-27 23:01:06.234760: I tensorflow/core/platform/cpu_feature_g

In [2]:
raw_dataset = pd.read_csv('linear-relationship.csv')
print('Done')

Done


In [3]:
dataset = raw_dataset.copy()
dataset.tail()

,x,y
9995,9995,19995
9996,9996,19997
9997,9997,19999
9998,9998,20001
9999,9999,20003


In [4]:
train_ds = dataset.sample(frac=0.8, random_state=0)
test_ds = dataset.drop(train_ds.index)
train_ds.tail()
print('--')
test_ds.tail()

--


,x,y
9963,9963,19931
9969,9969,19943
9972,9972,19949
9982,9982,19969
9995,9995,19995


In [5]:
train_ds.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
x,8000.0,5003.4595,2887.354272,1.0,2493.75,5006.5,7517.25,9999.0
y,8000.0,10011.9190,5774.708543,7.0,4992.50,10018.0,15039.50,20003.0


In [6]:
train_features = train_ds.copy()
test_features = test_ds.copy()

train_labels = train_features.pop('y')
test_labels = test_features.pop('y')

In [7]:
tsr_train_features = tf.convert_to_tensor(train_features)
tsr_train_labels = tf.convert_to_tensor(train_labels)

tsr_test_features = tf.convert_to_tensor(test_features)
tsr_test_labels = tf.convert_to_tensor(test_labels)

2024-02-27 23:01:08.186117: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-27 23:01:08.190164: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-02-27 23:01:08.193526: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [8]:
normalizer = tf.keras.layers.Normalization(axis=-1)
normalizer.adapt(np.array(tsr_train_features))

2024-02-27 23:01:08.584588: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type CPU is enabled.
2024-02-27 23:01:09.032452: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


In [9]:
a = np.array(train_features)

a_normalizer = layers.Normalization(input_shape=[1,], axis=None)
a_normalizer.adapt(a)
print ('Done')

Done


In [10]:
a_model = tf.keras.Sequential([
    a_normalizer,
    layers.Dense(units=1)
])

In [11]:
a_model.compile(optimizer=tf.keras.optimizers.Adam(), loss='mean_absolute_error')

print('Done')

Done


In [12]:
%%time
history = a_model.fit(
    tsr_train_features,
    tsr_train_labels,
    epochs=40,
    validation_split=0.2)

Epoch 1/40
  1/200 [..............................] - ETA: 3:11 - loss: 9215.9609

2024-02-27 23:01:11.065787: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f8bc151f4a0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-02-27 23:01:11.065823: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Quadro RTX 4000, Compute Capability 7.5
2024-02-27 23:01:11.071373: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-02-27 23:01:11.090013: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-02-27 23:01:11.169285: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


200/200 [==============================] - 1s 2ms/step - loss: 10058.8428 - val_loss: 9823.6279
Epoch 2/40
200/200 [==============================] - 0s 2ms/step - loss: 10058.6416 - val_loss: 9823.4277
Epoch 3/40
200/200 [==============================] - 0s 2ms/step - loss: 10058.4473 - val_loss: 9823.2285
Epoch 4/40
200/200 [==============================] - 0s 2ms/step - loss: 10058.2422 - val_loss: 9823.0283
Epoch 5/40
200/200 [==============================] - 0s 2ms/step - loss: 10058.0410 - val_loss: 9822.8291
Epoch 6/40
200/200 [==============================] - 0s 2ms/step - loss: 10057.8467 - val_loss: 9822.6289
Epoch 7/40
200/200 [==============================] - 0s 2ms/step - loss: 10057.6484 - val_loss: 9822.4297
Epoch 8/40
200/200 [==============================] - 0s 2ms/step - loss: 10057.4404 - val_loss: 9822.2305
Epoch 9/40
200/200 [==============================] - 0s 2ms/step - loss: 10057.2402 - val_loss: 9822.0303
Epoch 10/40
200/200 [===========================

In [13]:
a1 = [[11000]]
b1 = a_model.predict(a1)

1/1 [==============================] - 0s 93ms/step


In [14]:
b1

array([[10.072]], dtype=float32)

In [15]:
train_ds.describe().transpose()[['mean', 'std']]

,mean,std
x,5003.4595,2887.354272
y,10011.9190,5774.708543


In [16]:
# original_value = (y_pred * standard_deviation) + mean

original_value = (11.91 * 2887.354272) + 5003.4595
print(original_value)

39391.848879519996
